In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import plotly.express as px
import locale
from datetime import datetime

In [ ]:
# List all CSV files in a directory
csv_files = glob.glob('/kaggle/input/cyclistic-2022-jul2023/*.csv')

In [ ]:
# Initialize an empty list to store DataFrames
dfs = []

In [ ]:
# Load and append each CSV file to the list of DataFrames
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

In [ ]:
# Combine all DataFrames in the list using pd.concat()
df = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
# Check the columns
df.columns

In [ ]:
# Drop unnecessary columns
drop_columns = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
df = df.drop(columns=drop_columns)

In [ ]:
# Convert 'started time' and 'ended time' columns to datetime objects
df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Create a column that calculates the riding duration
df['duration'] = df['ended_at'] - df['started_at']

In [ ]:
# Create a column that converts the duration into hours spent
df['hours'] = df['duration'].dt.total_seconds() / 3600

In [ ]:
# Create day of the week column
df['day_of_week'] = df['started_at'].dt.day_name()

In [ ]:
# Define a function to map hours to time periods
def get_time_period(hour):
    if 4 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 20:
        return 'Evening'
    else:
        return 'Night'
    
# Extract the hour component from the date and map it to time periods
df['time_period'] = df['started_at'].dt.hour.apply(get_time_period)

In [ ]:
# Check for missing values in all columns
df.isna().sum()

In [ ]:
# Drop the null values
df = df.dropna()

In [ ]:
# Check whether the null values have been dropped
df.isna().sum()

In [ ]:
# Check for inconsitent riding duration (ended_at value is earlier than started_at)
filtered_rows = df[(df['member_casual'] == 'casual') & (df['duration'].dt.total_seconds() / 3600 < 0)].sort_values(by='duration', ascending=False)
filtered_rows['duration']

In [ ]:
# Remove rows with less than 0 second duration
df = df[df['duration'].dt.total_seconds() / 3600 >= 0]

In [ ]:
# Check whether the rows have been removed
df[(df['member_casual'] == 'casual') & (df['duration'].dt.total_seconds() / 3600 < 0)].sort_values(by='duration', ascending=False)

In [ ]:
# Extract Month-Year from 'started time'
df['Month-Year'] = df['started_at'].dt.strftime('%b-%Y')

In [ ]:
# Define the custom order for days of the week, time period, and month-year
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_period_order = ['Morning', 'Afternoon', 'Evening', 'Night']
month_sorted = sorted(df['Month-Year'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%Y'))


In [ ]:
# Convert 'day_of_week', 'time_period', and 'Month-Year' columns to categorical with the custom order
df['day_of_week'] = pd.Categorical(df['day_of_week'], categories=day_order, ordered=True)
df['time_period'] = pd.Categorical(df['time_period'], categories=time_period_order, ordered=True)
df['Month-Year'] = pd.Categorical(df['Month-Year'], categories=month_sorted, ordered=True)

In [ ]:
# Check the summary of the dataframe
df.info()

In [ ]:
# Count the number of riders by rider type
member_count = df.groupby('member_casual')['ride_id'].nunique()

# Create a pie chart to display Total Customers by Membership Status
plt.figure(figsize=(6, 6))

# Define custom colors
colors = ['#FF9999', '#66B2FF']

# Calculate percentages
percentages = (member_count / member_count.sum()) * 100

# Format numbers with commas for thousand separators
formatted_numbers = member_count.map('{:,}'.format)

# Create labels with both numbers and percentages
labels = [f'{formatted_numbers[i]} ({percentages[i]:.1f}%)' for i in range(len(member_count))]

# Define a custom function for autopct
def my_autopct(pct):
    return labels[int(pct / 100. * len(labels))]

# Create the pie chart with custom colors and bold text
wedges, texts, autotexts = plt.pie(member_count, labels=member_count.index, startangle=140, autopct=my_autopct,
                                    textprops={'color': 'white', 'weight': 'bold'}, colors=colors)

# Customize text styles for better visibility
for text, autotext in zip(texts, autotexts):
    text.set(size=12, color='white')
    autotext.set(size=12, color='black')

plt.title('Proportion of Riders by Membership Status', color='white', weight='bold')
plt.axis('equal')
plt.show()

In [ ]:
# Display the average riding duration by membership status in minutes
average_duration_by_member_type = df.groupby('member_casual')['hours'].mean() * 60

# Format the result to include 'mins' after the value
average_duration_by_member_type = average_duration_by_member_type.apply(lambda x: f'{x:.2f} mins')

average_duration_by_member_type

In [ ]:
# Count the number of riders by time period
time_period_count = df.groupby('time_period')['ride_id'].count().reset_index()

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot the pie chart
colors = ['gold', 'lightcoral', 'lightskyblue', 'lightgreen']
explode = (0.1, 0.1, 0.1, 0.1)  # Explode the highest percentage slice
ax1.pie(time_period_count['ride_id'], explode=explode, labels=time_period_count['time_period'], colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
ax1.axis('equal')  # Set the aspect ratio to be equal

# Find the 'time_period' with the highest percentage
max_percentage_time_period = time_period_count.loc[time_period_count['ride_id'].idxmax(), 'time_period']

# Filter data for the highest percentage 'time_period' and group by 'member_type'
highest_percentage_data = df[df['time_period'] == max_percentage_time_period]
member_type_counts = highest_percentage_data.groupby('member_casual')['ride_id'].count()

# Calculate the percentages for the breakdown
total_count = member_type_counts.sum()
percentage_values = (member_type_counts / total_count) * 100

# Plot the breakdown as horizontal bars
bars = ax2.barh(percentage_values.index, percentage_values.values, color=colors)
for bar, percentage in zip(bars, percentage_values):
    ax2.text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2, f'{percentage:.1f}%', va='center')

# Create ConnectionPatches to connect the pie chart to the breakdown
for i, member_type in enumerate(member_type_counts.index):
    xyA = (1.1, i)  # Starting point on the right of the pie chart
    xyB = (0, i)    # Ending point on the left of the breakdown
    con = ConnectionPatch(xyA=xyA, xyB=xyB, coordsA="data", coordsB="data",
                          axesA=ax1, axesB=ax2, color=colors[i])
    ax2.add_artist(con)

# Set labels and title for the breakdown
ax2.set_xlabel('Percentage (%)')
ax2.set_ylabel('Member Type')
ax2.set_title(f"Breakdown of {max_percentage_time_period} by Member Type")

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Filter the DataFrame for 'Morning' time_period
morning_df = df[df['time_period'] == 'Morning']

# Group by 'member_casual' and calculate the count of 'ride_id'
member_casual_counts = morning_df.groupby('member_casual')['ride_id'].count().reset_index()

# Calculate the total riders for all categories
total_riders = member_casual_counts['ride_id'].sum()

# Format the count of 'ride_id' with thousand separator
member_casual_counts['Total_Riders'] = member_casual_counts['ride_id'].apply(lambda x: '{:,}'.format(x))

# Calculate the percentage of riders for each category and round to the nearest integer
member_casual_counts['%ofRiders'] = member_casual_counts['ride_id'].apply(lambda x: str(round((x / total_riders) * 100))) + '%'

# Drop the original 'ride_id' column
member_casual_counts = member_casual_counts.drop(columns='ride_id')

# Rename columns for display
member_casual_counts.columns = ['Member_Casual', 'Total_Riders', '%ofRiders']

# Display the tabular result
print(member_casual_counts)

In [ ]:
# Create a pivot table with 'Month-Year' as index, 'time_period' as columns, and 'hours' as values
pt2 = df.pivot_table(index='Month-Year', columns='member_casual', values='hours', aggfunc='sum')

In [ ]:
# Plot the bar chart
ax = pt2.plot(kind='bar', stacked=True, figsize=(10, 6))

# Set labels and title
plt.xlabel('Year-Month')
plt.ylabel('Ride Length (hrs)')
plt.title('Riding Trend by Year-Month')

# Add legend
plt.legend(title='Rider Type')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Set the locale to use a comma as the thousands separator
locale.setlocale(locale.LC_ALL, '')

# Create a pivot table with 'member_casual' and 'rideable_type' as rows, 'day_of_week' as columns, and 'hours' as values
pt1 = pd.pivot_table(df, values='hours', index=['member_casual', 'rideable_type'], columns='day_of_week', aggfunc='sum')

# Format the values to show only 2 decimal places
pt1_formatted = pt1.applymap(lambda x: f'{x:,.2f}')

# Print the result
pt1_formatted

In [ ]:
# Group the data by 'Day_Of_Week' and 'member_type' and count unique 'ride_id'
grouped = df.groupby(['day_of_week', 'member_casual']).size().unstack(fill_value=0)
total_counts = grouped.sum(axis=1)
percentage_data = grouped.divide(total_counts, axis=0) * 100

In [ ]:
# Create a 100% stacked bar chart
ax = percentage_data.plot(kind='bar', stacked=True, figsize=(10, 6))
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(f'{height:.1f}%', (x + width / 2, y + height / 2), ha='center', va='center', fontsize=10, color='white')
plt.xlabel('Day of Week')
plt.ylabel('Percentage of Riders')
plt.title('100% Stacked Bar Chart by Day of Week and Rider Type')
plt.legend(title='Membership Type')
plt.tight_layout()
plt.show()

In [ ]:
df[df['member_casual'] == 'casual'].groupby('start_station_name')['ride_id'].count().sort_values(ascending=False).head(10).apply(lambda x: f'{x:,}')

In [ ]:
df[df['member_casual'] == 'member'].groupby('start_station_name')['ride_id'].count().sort_values(ascending=False).head(10).apply(lambda x: f'{x:,}')

In [ ]:
# Group by 'Month-Year', 'rideable_type' and count the 'ride_id'
grouped = df.groupby(['Month-Year', 'rideable_type'])['ride_id'].count().unstack(fill_value=0)

# Create the trendline plot
plt.figure(figsize=(10, 6))

for rideable_type in grouped.columns:
    plt.plot(grouped.index, grouped[rideable_type], marker='o', label=rideable_type)

plt.title('Bike Trends Overtime')
plt.ylabel('Number of Riders')
plt.legend(title='Rideable Type', loc='upper right')
plt.grid(True)

# Define custom X-axis tick positions and labels
custom_ticks = ['Jan-2022', 'Apr-2022', 'Jul-2022', 'Oct-2022', 'Jan-2023', 'Apr-2023', 'Jul-2023']
custom_labels = [grouped.index.get_loc(date) for date in custom_ticks]

plt.xticks(custom_labels, custom_ticks, rotation=45)

plt.show()

## Conclusion

In light of the analysis provided earlier, several noteworthy observations can be made:
1. Casual riders tend to complete fewer trips but display a penchant for longer rides (1.93x more minutes)

2. The number of riders is notably higher during weekends when compared to weekdays

3. The majority of rides undertaken by members can be due to their their daily commuting needs, resulting in a substantial weekday rider distribution with a ratio of approximately 7:3. This observation is further supported by the fact that 70% of riders during morning hours are members.

4. Conversely, casual riders predominantly opt for weekend rides, as exemplified by the preference for popular tourist destinations in their top riding locations

5. The period from November to March exhibits a decline in riders, likely attributed to the onset of winter conditions. Conversely, a surge of riders are observed during the summer months, with peak activity occurring in July.


Taking these insights into consideration, here are some recommendations to attract and convert more riders into members:
1. Promote Weekend Member Benefits: Given the higher weekend rider volume, emphasize the advantages of becoming a member during weekends. Highlight special weekend discounts, exclusive access, or incentives that can make membership more appealing during these peak riding times.

2. Seasonal Promotions: Capitalize on the seasonal trends in cycling. During the winter months when riders tends to decrease, promote membership benefits that encourage riders to stay engaged, such as winter-specific perks. During peak summer months, launch campaigns that highlight the advantages of being a member during the busiest riding season. To save cost, we can opt to perform marketing campaigns during busy months.

3. Tailored Membership Packages: Develop membership packages that cater to the commuting needs of riders. Offer flexible pricing options, monthly plans, or discounted rates for frequent commuters, making membership a cost-effective choice for their daily transportation.
